# Getting Started with Pilot-Streaming and Quantum Computing (on local environment or remote VM via ssh)

In the first step we need to import all required packages and modules into the Python Path

Pilot-Streaming can be used to manage the Dask and Kafka environments both in the cloud and on the edge. 



`resource`: URL of the Local Resource Manager. Examples:

* `slurm://localhost`: Submit to local SLURM resource manager, e.g. on master node of Wrangler or Stampede
* `slurm+ssh://login1.wrangler.tacc.utexas.edu`: Submit to Wrangler master node SLURM via SSH (e.g. on node running a job)
* `os://` Openstack
* `ec2://` EC2


`type:` The `type` attributes specifies the cluster environment. It can be: `Spark`, `Dask` or `Kafka`.

***For standard task-level parallelism `Dask` is recommended.***


Depending on the resource there might be other configurations necessary, e.g. to ensure that the correct subnet is used the Spark driver can be configured using various environment variables:   os.environ["SPARK_LOCAL_IP"]='129.114.58.2'



In [24]:
# Pilot-Streaming
import os, sys
sys.path.insert(0, os.path.abspath('../..'))
import distributed
import json
import pilot.streaming
import getpass
import socket
import logging
sys.modules['pilot.streaming']

#RESOURCE_URL_HPC="slurm+ssh://login4.stampede2.tacc.utexas.edu"
RESOURCE_URL_HPC="ssh://localhost"
WORKING_DIRECTORY=os.path.join(os.environ["HOME"], "work")


# 1. Dask on local machine

In [25]:
pilot_compute_description_dask = {
    "resource":"ssh://{}@localhost".format(getpass.getuser()),
    "working_directory": os.path.join(os.path.expanduser("~"), "work"),
    "number_cores": 48,
    "queue": "normal",
    "walltime": 59,
    "type":"dask"
}

In [ ]:
##%%time
dask_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description_dask)
dask_pilot.wait()
dask_pilot.get_details()

## Test edge dask behind firewall

In [27]:
dask_client  = distributed.Client(dask_pilot.get_details()['master_url'])
#dask_client  = distributed.Client()
dask_client.scheduler_info()

12/27/2022 12:04:11 PM - root - DEBUG - Results of scheduler startup file check: True
12/27/2022 12:04:11 PM - asyncio - DEBUG - Using selector: KqueueSelector


{'type': 'Scheduler',
 'id': 'Scheduler-a1ce4cc5-df17-45c2-9621-0c92bbd90fc9',
 'address': 'tcp://192.168.0.100:49418',
 'services': {'dashboard': 49417},
 'started': 1672139049.903553,
 'workers': {}}

In [28]:
dask_client.gather(dask_client.map(lambda a: a*a, range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [29]:
dask_client.gather(dask_client.map(lambda a: socket.gethostname(), range(10)))

['macbook',
 'macbook',
 'macbook',
 'macbook',
 'macbook',
 'macbook',
 'macbook',
 'macbook',
 'macbook',
 'macbook']

# Create simple quantum circuit for simulation

Requirements:

        pip install pennylane 

In [30]:
import pennylane as qml
from timeit import default_timer as timer


def run_circuit():
    wires = 4
    layers = 1
    num_runs = 50
    GPUs = 1

    dev = qml.device('default.qubit', wires=wires, shots=None)

    @qml.qnode(dev)
    def circuit(parameters):
        qml.StronglyEntanglingLayers(weights=parameters, wires=range(wires))
        return [qml.expval(qml.PauliZ(i)) for i in range(wires)]

    shape = qml.StronglyEntanglingLayers.shape(n_layers=layers, n_wires=wires)
    weights = qml.numpy.random.random(size=shape)
    val = circuit(weights)
    return val

In [31]:
dask_client.gather(dask_client.map(lambda a: run_circuit(), range(10)))

[array([0.39116895, 0.55268237, 0.34928422, 0.31240458]),
 array([0.49618673, 0.62145163, 0.43799797, 0.41511899]),
 array([0.40579505, 0.7561093 , 0.50686487, 0.39688098]),
 array([0.56276997, 0.79791631, 0.72717203, 0.51695379]),
 array([0.39395371, 0.99843012, 0.66850593, 0.393824  ]),
 array([0.48526631, 0.44611991, 0.44565011, 0.39794519]),
 array([0.76769033, 0.71596752, 0.56223612, 0.55378307]),
 array([0.556863  , 0.8163597 , 0.55179448, 0.54099155]),
 array([0.489417  , 0.45852117, 0.43769538, 0.28793255]),
 array([0.63178445, 0.93206321, 0.69866833, 0.58899336])]